In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from tqdm import tqdm
from functools import partial
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, explained_variance_score
import h3
from IPython.display import clear_output
from datetime import datetime
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Introduction - An Artificial Salmon Brain?

The idea here is really simple. We want to build a model of how Chinook salmon move about and we're going to give it a go by thinking about choice. 

At each timestep we happen to know where our fish is and therefore (given some maximum travel distance) where it could go. This means our fish is presented with some number of choices. 
For each of these choices we can imagine capturing whether the fish chose it or not and accumulating all these decisions over time and over many different fish. Then we could ask the 
following question:

If my fish is presented with a new option, what are the odds it would pick it? 

If we can model this, then in some sense we can model the distribution of "utility" the fish sees across its options. Then in simulation we can simply iterate over all of these
options, calculate the utility, and then randomly pick with a weight given by the utility of each possible choice. 

Let's give it a go.

# Loading up the Tracks

In [2]:
data = pd.read_csv("data/tag_tracks.csv").rename({
    "Ptt": "ptt",
    "Latitude": "lat",
    "Longitude": "lon",
    "Dates - Date Key → Date": "date",
    "Dates - Date Key → Year": "year",
    "Dates - Date Key → Month": "month",
    "Dates - Date Key → Day": "day",
}, axis=1)
print(data.shape)
data.head()

(7532, 7)


,ptt,lat,lon,date,year,month,day
0,129843,54.131760,-166.922615,2013-12-19,2013,12,19
1,129843,54.258072,-166.884086,2013-12-20,2013,12,20
2,129843,54.312433,-166.910525,2013-12-21,2013,12,21
3,129843,54.358280,-166.817057,2013-12-22,2013,12,22
4,129843,54.389694,-166.676901,2013-12-23,2013,12,23


# Discretizing the Options in Space

In [3]:
def find_neighbors(h3_index, threshold_km, neighbors_index):
    h3_coords = h3.h3_to_geo(h3_index)
    checked = set()
    neighbors = set()
    distance = 1
    found_neighbors = True
    while found_neighbors:
        found_neighbors = False
        candidates = h3.k_ring(h3_index, distance)
        new_candidates = set(candidates) - checked
        for candidate in new_candidates:
            if geopy.distance.geodesic(h3_coords, h3.h3_to_geo(candidate)).km <= threshold_km:
                neighbors.add(candidate)
                found_neighbors = True
            checked.add(candidate)
        distance += 1
    neighbors_index[h3_index] = neighbors



RESOLUTION = 4
MAX_KM = 100

selected_h3 = set([
    h3.geo_to_h3(lat, lon, RESOLUTION)
    for lat, lon in zip(data['lat'], data['lon'])
])

neighbors_index = {}
for h3_index in tqdm(selected_h3):
    find_neighbors(h3_index, MAX_KM, neighbors_index)

100%|██████████| 738/738 [00:02<00:00, 247.11it/s]


# Capturing Options and Relevant MetaData

In order to capture the options available at each timestep as well as the "momentum" of the fish we're going to form our tracks into triplets.

In [4]:
def get_direction(lat1, lon1, lat2, lon2):
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    size = np.linalg.norm([dlat, dlon])
    dlat = dlat / size if size > 0 else np.nan 
    dlon = dlon / size if size > 0 else np.nan
    return dlat, dlon

In [5]:
def create_triplets(data, neighbors_index):
    triplets = []
    last_direction_lat = 0
    last_direction_lon = 0
    for ptt in tqdm(data["ptt"].unique()):
        rows = [row for _, row in data[data["ptt"] == ptt].sort_values("date", ascending=True).iterrows()]
        for start, middle, end in zip(rows[:-2], rows[1:-1], rows[2:]):
            middle_lat, middle_lon = h3.h3_to_geo(h3.geo_to_h3(middle['lat'], middle['lon'], RESOLUTION))
            start_lat, start_lon = h3.h3_to_geo(h3.geo_to_h3(start['lat'], start['lon'], RESOLUTION))
            direction_lat, direction_lon = get_direction(start_lat, start_lon, middle_lat, middle_lon)
            triplets.append({
                "ptt": ptt,
                "start_date": middle["date"],
                "end_date": end["date"],
                "start_lat": middle["lat"],
                "start_lon": middle["lon"],
                "end_lat": end["lat"],
                "end_lon": end["lon"],
                "distance": geopy.distance.geodesic(
                    (start["lat"], start["lon"]), (end["lat"], end["lon"])
                ).km,
                "prior_direction_lat": direction_lat if np.isfinite(direction_lat) else last_direction_lat,
                "prior_direction_lon": direction_lon if np.isfinite(direction_lon) else last_direction_lon,
                "remained": not np.isfinite(direction_lat) and not np.isfinite(direction_lon)
            })
            last_direction_lat = direction_lat if np.isfinite(direction_lat) else last_direction_lat
            last_direction_lon = direction_lon if np.isfinite(direction_lon) else last_direction_lon
    triplets = pd.DataFrame(triplets)
    return triplets
    
triplets = create_triplets(data, neighbors_index)    
print(triplets.shape)
triplets.head()

100%|██████████| 111/111 [00:01<00:00, 95.78it/s] 


(7310, 11)


,ptt,start_date,end_date,start_lat,start_lon,end_lat,end_lon,distance,prior_direction_lat,prior_direction_lon,remained
0,129843,2013-12-20,2013-12-21,54.258072,-166.884086,54.312433,-166.910525,20.126011,0.988573,0.150745,False
1,129843,2013-12-21,2013-12-22,54.312433,-166.910525,54.358280,-166.817057,11.977147,0.988573,0.150745,True
2,129843,2013-12-22,2013-12-23,54.358280,-166.817057,54.389694,-166.676901,17.456323,0.988573,0.150745,True
3,129843,2013-12-23,2013-12-24,54.389694,-166.676901,54.407429,-166.526618,19.647607,0.988573,0.150745,True
4,129843,2013-12-24,2013-12-25,54.407429,-166.526618,54.407463,-166.528348,9.848835,0.988573,0.150745,True


# Splitting the Data

We need to split into training and testing sets to evaluate how much overfitting we're experiencing.

In [6]:
training_ptt = set(
    np.random.choice(data['ptt'].unique(), round(data['ptt'].unique().shape[0] * 0.7), replace=False)
)
testing_ptt = set(data['ptt'].unique()) - training_ptt
print(len(training_ptt), len(testing_ptt))

78 33


# Finding the Options

Next we'll find the options available to the fish and annotate them with things like, what direction would this move be, and would this be a move at all?

In [7]:
def create_options_from_triplets(triplets):
    rows = []
    choice_id = -1
    for _, row in tqdm(triplets.iterrows()):
        choice_id += 1
        start = h3.geo_to_h3(row['start_lat'], row['start_lon'], RESOLUTION)
        end = h3.geo_to_h3(row['end_lat'], row['end_lon'], RESOLUTION)
        date = datetime.strptime(row['end_date'], '%Y-%m-%d')
        
        start_lat, start_lon = h3.h3_to_geo(start)
        end_lat, end_lon = h3.h3_to_geo(end)
        direction_lat, direction_lon = get_direction(start_lat, start_lon, end_lat, end_lon)
        rows.append({
            'h3_index': end, 
            'chosen': 1,
            'date': date,
            'year': date.year,
            'month': date.month,
            'day': date.day,
            'prior_direction_lat': row['prior_direction_lat'],
            'prior_direction_lon': row['prior_direction_lon'],
            'direction_lat': direction_lat if np.isfinite(direction_lat) else 0,
            'direction_lon': direction_lon if np.isfinite(direction_lon) else 0,
            'remained_before': row['remained'],
            'remain_now': not np.isfinite(direction_lat) and not np.isfinite(direction_lon),
            'choice_id': choice_id
        })
        for neighbor in neighbors_index[start]:
            end_lat, end_lon = h3.h3_to_geo(neighbor)
            direction_lat, direction_lon = get_direction(start_lat, start_lon, end_lat, end_lon)
            if neighbor != end:
                rows.append({
                    'h3_index': neighbor, 
                    'chosen': 0,
                    'date': date,
                    'year': date.year,
                    'month': date.month,
                    'day': date.day,
                    'prior_direction_lat': row['prior_direction_lat'],
                    'prior_direction_lon': row['prior_direction_lon'],
                    'direction_lat': direction_lat if np.isfinite(direction_lat) else 0,
                    'direction_lon': direction_lon if np.isfinite(direction_lon) else 0,
                    'remained_before': row['remained'],
                    'remain_now': not np.isfinite(direction_lat) and not np.isfinite(direction_lon),
                    'choice_id': choice_id
                })
    return pd.DataFrame(rows)

train = create_options_from_triplets(triplets[triplets['ptt'].isin(training_ptt)])
test = create_options_from_triplets(triplets[triplets['ptt'].isin(testing_ptt)])

5212it [00:01, 3224.73it/s]
2098it [00:00, 3768.64it/s]


# Loading in Environmental Data

We'll now load in bathymetry, sea surface temps (avg per month over a year), and a measure of primary productivity (also avg per month over a year).

In [8]:
def spatial_key_to_index(spatial_key):
    return hex(spatial_key)[2:]

elevation = pd.read_csv("data/bathymetry.csv")
elevation['h3_index'] = elevation['h3_index'].astype(np.int64).astype(str)
elevation['h3_index'] = elevation.apply(lambda row: spatial_key_to_index(np.int64(row['h3_index'])), axis=1)
elevation.head()

,h3_index,elevation
0,8402105ffffffff,-450.010400
1,840210dffffffff,-1184.517832
2,8402121ffffffff,-499.541915
3,8402123ffffffff,-270.425039
4,8402125ffffffff,-1022.894455


In [9]:
surface_temps = pd.read_csv("data/surface_temps.csv").rename({
    "H3 Key 4": "h3_index",
    "Dates - Date Key → Month": "month",
    "Dates - Date Key → Day": "day",
    "Temperature C": "temp"
}, axis=1)[['h3_index', 'month', 'day', 'temp']]
surface_temps['h3_index'] = surface_temps['h3_index'].astype(np.int64).astype(str)
surface_temps['h3_index'] = surface_temps.apply(lambda row: spatial_key_to_index(np.int64(row['h3_index'])), axis=1)
surface_temps.head()

,h3_index,month,day,temp
0,8402101ffffffff,1,15,-0.2685
1,8402103ffffffff,1,15,-0.2685
2,8402105ffffffff,1,15,-0.2685
3,8402107ffffffff,1,15,-0.2685
4,8402109ffffffff,1,15,-0.2685


In [10]:
surface_temps.day.unique(), surface_temps.month.unique()

(array([15]), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]))

In [11]:
primary_productivity = pd.read_csv("data/primary_productivity.csv").rename({
    "H3 Key 4": "h3_index",
    "Dates - Date Key → Month": "month",
    "Dates - Date Key → Day": "day",
    "Log Chla Ave": "log_chla"
}, axis=1)[['h3_index', 'month', 'day', 'log_chla']]
primary_productivity['h3_index'] = primary_productivity['h3_index'].astype(np.int64).astype(str)
primary_productivity['h3_index'] = primary_productivity.apply(lambda row: spatial_key_to_index(np.int64(row['h3_index'])), axis=1)
primary_productivity.loc[primary_productivity['log_chla'] < 3, 'log_chla'] = 3
primary_productivity.head()

,h3_index,month,day,log_chla
0,840c401ffffffff,1,15,6.683674
1,840c407ffffffff,1,15,6.726833
2,840c409ffffffff,1,15,6.403988
3,840c40dffffffff,1,15,6.067268
4,840c411ffffffff,1,15,5.836758


In [12]:
primary_productivity.day.unique(), primary_productivity.month.unique()

(array([15]), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]))

# Building our Features

Now we can join everything together! Note we're adding a dot product which should give us an idea of how choosing a specific cell is in keeping with the overall momentum.

In [13]:
def add_features(base_data):
    base_data = base_data[['h3_index', 'chosen', 'date', 'year', 'month', 'day', 'prior_direction_lat', 'prior_direction_lon', 'direction_lat', 'direction_lon', 'choice_id', 'remained_before', 'remain_now']]

    base_data = base_data.merge(elevation, on='h3_index')
    base_data = base_data.merge(surface_temps[['h3_index', 'month', 'temp']], on=['h3_index', 'month'])
    base_data = base_data.merge(primary_productivity[['h3_index', 'month', 'log_chla']], on=['h3_index', 'month'])

    base_data['dot_product'] = base_data['prior_direction_lat'] * train['direction_lat'] + train['prior_direction_lon'] * train['direction_lon']

    base_data['lat'] = base_data.apply(lambda row: h3.h3_to_geo(row['h3_index'])[0], axis=1)
    base_data['lon'] = base_data.apply(lambda row: h3.h3_to_geo(row['h3_index'])[1], axis=1)
    return base_data

train = add_features(train)
test = add_features(test)
train.head()

,h3_index,chosen,date,year,month,day,prior_direction_lat,prior_direction_lon,direction_lat,direction_lon,choice_id,remained_before,remain_now,elevation,temp,log_chla,dot_product,lat,lon
0,8422d03ffffffff,1,2013-12-21,2013,12,21,0.988573,0.150745,0.0,0.0,0,False,True,-771.670639,1.215667,8.805272,0.000000,54.360925,-166.742418
1,8422d03ffffffff,1,2013-12-22,2013,12,22,0.988573,0.150745,0.0,0.0,1,True,True,-771.670639,1.215667,8.805272,0.350207,54.360925,-166.742418
2,8422d03ffffffff,1,2013-12-23,2013,12,23,0.988573,0.150745,0.0,0.0,2,True,True,-771.670639,1.215667,8.805272,-0.358522,54.360925,-166.742418
3,8422d03ffffffff,1,2013-12-24,2013,12,24,0.988573,0.150745,0.0,0.0,3,True,True,-771.670639,1.215667,8.805272,-1.000000,54.360925,-166.742418
4,8422d03ffffffff,1,2013-12-25,2013,12,25,0.988573,0.150745,0.0,0.0,4,True,True,-771.670639,1.215667,8.805272,0.032613,54.360925,-166.742418


# Training a Model

We'll use elevation, temp, log_chla, our dot product (momentum term), and whether the choice would be staying.

We'll also use some cross validation to ensure we don't overfit to our data.

In [14]:
features = ['elevation', 'temp', 'log_chla', 'dot_product', 'remain_now']

X = train[features]
y = train['chosen']

models = {
    "Random Forest": RandomForestRegressor(
        random_state=42, n_jobs=3
    )
}
param_grids = {
    "Random Forest": {"n_estimators": [10, 20, 50, 100], "min_samples_leaf": [20, 50, 100, 200]}
}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
for name, model in models.items():
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        return_train_score=True,
        cv=cv,
        refit=True,
    ).fit(X, y)
    result = {"model": name, "cv_results": pd.DataFrame(grid_search.cv_results_), "best_estimator": grid_search.best_estimator_}
    results.append(result)

In [15]:
results[0]["best_estimator"].n_estimators, results[0]["best_estimator"].min_samples_leaf

(100, 100)

# How'd we Do?

Looks like we explained just over 40% of the variance in our data. Pretty good!

In [16]:
train["pred"] = results[0]["best_estimator"].predict(train[features])
train["error"] = train["chosen"] - train["pred"]
print(mean_squared_error(train["chosen"], train["pred"]))
print(explained_variance_score(train["chosen"], train["pred"]))

0.027658026010781205
0.44323562544131934


In [17]:
test["pred"] = results[0]["best_estimator"].predict(test[features])
test["error"] = test["chosen"] - test["pred"]
print(mean_squared_error(test["chosen"], test["pred"]))
print(explained_variance_score(test["chosen"], test["pred"]))

0.027113854044160036
0.46002160664032177


# Simulating it All

We'll now create tracks that are:

1. *random*: totally random walk
2. *fake*: random walk using our model's utility predictions
3. *real*: the real deal

In [18]:
ptts = np.random.choice(data['ptt'].unique(), data['ptt'].nunique(), replace=False)

In [19]:
full_features = elevation
full_features = full_features.merge(surface_temps[['h3_index', 'month', 'temp']], on=['h3_index'])
full_features = full_features.merge(primary_productivity[['h3_index', 'month', 'log_chla']], on=['h3_index', 'month'])
full_features.head()

,h3_index,elevation,month,temp,log_chla
0,8402123ffffffff,-270.425039,5,-0.268500,3.000000
1,8402123ffffffff,-270.425039,6,0.531628,3.000000
2,8402801ffffffff,-2262.651136,6,1.136333,3.018073
3,8402803ffffffff,-916.098408,6,0.785190,3.000000
4,8402805ffffffff,-2577.574399,6,0.903583,3.061052


In [20]:
def create_random_track(lat, lon, date, model, steps):
    path = [(lat, lon, date)]
    last_prior_direction_lat = np.nan
    last_prior_direction_lon = np.nan
    for _ in range(steps):
        h3_index = h3.geo_to_h3(lat, lon, RESOLUTION)
        if h3_index not in neighbors_index:
            find_neighbors(h3_index, MAX_KM, neighbors_index)
        raw_possibilities = list(neighbors_index[h3_index])
        possibilities = pd.DataFrame(raw_possibilities, columns=['h3_index'])
        possibilities['month'] = date.month
        possibilities = possibilities.merge(full_features, on=['h3_index', 'month'])
        if possibilities.shape[0] == 0:
            break

        if model is not None:
            if len(path) > 1:
                prior_direction_lat, prior_direction_lon = get_direction(path[-1][0], path[-1][1], path[-2][0], path[-2][1])
                if not np.isfinite(prior_direction_lat) or not np.isfinite(prior_direction_lon):
                    prior_direction_lat = last_prior_direction_lat
                    prior_direction_lon = last_prior_direction_lon
            else:
                prior_direction_lat = last_prior_direction_lat
                prior_direction_lon = last_prior_direction_lon

            last_prior_direction_lat = prior_direction_lat
            last_prior_direction_lon = prior_direction_lon

            possibilities['prior_direction_lat'] = prior_direction_lat if np.isfinite(prior_direction_lat) else 0
            possibilities['prior_direction_lon'] = prior_direction_lon if np.isfinite(prior_direction_lon) else 0

            possibilities['start_lat'] = lat
            possibilities['start_lon'] = lon
            possibilities['end_lat'] = possibilities.apply(lambda row: h3.h3_to_geo(row['h3_index'])[0], axis=1)
            possibilities['end_lon'] = possibilities.apply(lambda row: h3.h3_to_geo(row['h3_index'])[1], axis=1)

            possibilities['direction_lat'] = possibilities.apply(lambda r: get_direction(r['start_lat'], r['start_lon'], r['end_lat'], r['end_lon'])[0], axis=1).fillna(0)
            possibilities['direction_lon'] = possibilities.apply(lambda r: get_direction(r['start_lat'], r['start_lon'], r['end_lat'], r['end_lon'])[1], axis=1).fillna(0)

            possibilities['remained_before'] = (possibilities['prior_direction_lat'] == 0) & (possibilities['prior_direction_lon'] == 0)
            possibilities['remain_now'] = (possibilities['direction_lat'] == 0) & (possibilities['direction_lon'] == 0)
            possibilities['dot_product'] = possibilities['prior_direction_lat'] * possibilities['direction_lat'] + possibilities['prior_direction_lon'] * possibilities['direction_lon']

            X = possibilities[features]
            probs = model.predict(X)
            probs = probs / probs.sum()
            raw_possibilities = list(possibilities['h3_index'])
        else:
            probs = np.ones(len(raw_possibilities)) / len(raw_possibilities)
        
        selection = np.random.choice(range(len(raw_possibilities)), p=probs)
        lat, lon = h3.h3_to_geo(raw_possibilities[selection])
        date = date + pd.Timedelta(days=1)
        path.append((lat, lon, date))
    return pd.DataFrame(path, columns=['lat', 'lon', 'date'])

paths = []
for ptt in tqdm(ptts):
    df = data[data['ptt'] == ptt].sort_values("date", ascending=True).iloc[[0]]
    lat = df['lat'].values[0]
    lon = df['lon'].values[0]
    date = datetime.strptime(df['date'].values[0], '%Y-%m-%d')
    path = create_random_track(
        lat, lon, date,
        results[0]['best_estimator'],
        data[data['ptt'] == ptt].shape[0]
    )
    path['case'] = 'fake'
    path['ptt'] = ptt

    random_path = create_random_track(
        lat, lon, date,
        None,
        data[data['ptt'] == ptt].shape[0]
    )
    random_path['case'] = 'random'
    random_path['ptt'] = ptt

    old_path = data[data['ptt'] == ptt][['lat', 'lon', 'date']]
    old_path['date'] = pd.to_datetime(old_path['date'])
    old_path['h3_index'] = old_path.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], RESOLUTION), axis=1)
    old_path['lat'] = old_path.apply(lambda row: h3.h3_to_geo(row['h3_index'])[0], axis=1)
    old_path['lon'] = old_path.apply(lambda row: h3.h3_to_geo(row['h3_index'])[1], axis=1)
    old_path['case'] = 'real'
    old_path['ptt'] = ptt
    paths.append(pd.concat([path, old_path, random_path]))
    clear_output()

100%|██████████| 111/111 [02:17<00:00,  1.24s/it]


Note that sometimes we don't have features for a cell and so our tracks will be short. Let's just eliminate those for now for more of an apples to apples comparison.

In [21]:
df = pd.concat(paths)
d = df[df['case'] == 'fake'].groupby('ptt').agg({'lat': 'count'}).reset_index().merge(
    df[df['case'] == 'real'].groupby('ptt').agg({'lat': 'count'}).reset_index(),
    on='ptt'
).rename({'lat_x': 'fake', 'lat_y': 'real'}, axis=1)
d = d[d['fake'] / d['real']  > 0.8]
df = df[df['ptt'].isin(d['ptt'])]

## The Real Tracks

In [22]:
fig = px.scatter_geo(df[df['case'] == 'real'], lat='lat', lon='lon', color='ptt')
fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))

## A True Random Walk

In [23]:
fig = px.scatter_geo(df[df['case'] == 'random'], lat='lat', lon='lon', color='ptt')
fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))

## A Utility Guided Random Walk

In [24]:
fig = px.scatter_geo(df[df['case'] == 'fake'], lat='lat', lon='lon', color='ptt')
fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))

# Thoughts

So this definitely is headed in the right direction. Our simulated tracks are now limited to the same space that real salmon explore. However, what becomes immediately obvious is the fact that our simulated tracks are getting caught up in themselves and aren't moving around as much as the real tracks. Whether this is just the result of us not having some notion of "migration" or whether we really are not capturing something about foraging behavior is unclear. But that's definitely the next component to solve. 

Also there's definitely some kind of bug still lurking around as a few of the locations where fish were released just result in dead end tracks in our simulation. So at the very least I'd like to:

1. Find the bug.
2. Sort out why the model is not creating longer tracks (and fix it if I can).

Finally I think it's worth noting that I've gone down this "choice based utility" track in order to keep things general enough to apply to any odd creature out there. In theory if this approach works here it should be able to work for just about any fish we have tracking data for because it is just about choices, environmental data, and general notions of momentum. 

Until next time!